### How to compare files and database output will 10 lakhs file data in python

```
Steps:
1) Read file in chunks to handle large file sizes.
2) Fetch data from PostgreSQL in chunks using LIMIT and OFFSET.
3) Compare using sets for efficient lookup and difference detection.
4) Efficiently handle memory by processing both file and database in smaller parts.
```


In [16]:
# 1. Set Up the PostgreSQL Connection
# You already have the connection established, so make sure it's connected properly.

import psycopg2

# Connect to PostgreSQL
con = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='postgres',
    port=5432,
    dbname='db1'
)

# Create a cursor object
cur = con.cursor()

In [64]:
# 2. Open the Large File
# Open the file and read it in chunks (since it’s large). If it's a plain text file, handle each line, or use pandas if it's CSV.
def read_file_in_chunks(filename='BIGFILES.txt', chunk_size=100000):
    with open(filename, 'r') as file:
        chunk = []
        for i, line in enumerate(file):
            chunk.append(line.strip())
            if (i + 1) % chunk_size == 0:
                yield chunk  # Corrected: yield the current chunk
        if chunk:  # Yield any remaining lines in the last chunk
            yield chunk
for chunk in read_file_in_chunks():
    print(chunk)  # Process each chunk of lines


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [60]:
# 3. Fetch PostgreSQL Data in Chunks
# Use SQL to fetch rows in chunks from your PostgreSQL database. Here, we use LIMIT and OFFSET to control the amount of data fetched at a time.
def fetch_db_data_in_chunks(cursor, query, chunk_size=100000):
    offset = 0
    while True:
        cursor.execute(query + f" LIMIT {chunk_size} OFFSET {offset}")
        data = cursor.fetchall()
        if not data:
            break
        yield data
        offset += chunk_size

In [40]:
# 4. Compare File Data with Database Data
# Convert each chunk from the file and the database into sets and compare them.

def compare_file_and_db(file_chunk, db_chunk):
    file_data = set(file_chunk)  # Assume file_chunk is a list of strings or relevant entries
    db_data = set([row[0] for row in db_chunk])  # Assuming row[0] has the data to compare
    print(f"File data set: {file_data}")  # Debug: Print file set
    print(f"DB data set: {db_data}")  # Debug: Print DB set
    return file_data.symmetric_difference(db_data)  # Find differences

In [38]:
# 5. Combine Everything and Execute
# Read file chunks, fetch database chunks, and compare them.
def compare_large_file_with_db(file_path, db_connection, query):
    cursor = db_connection.cursor()

    try:
        for file_chunk in read_file_in_chunks(file_path):
            print(f"File chunk: {file_chunk}")  # Debug: Check file chunk content
            for db_chunk in fetch_db_data_in_chunks(cursor, query):
                print(f"DB chunk: {db_chunk}")  # Debug: Check DB chunk content
                differences = compare_file_and_db(file_chunk, db_chunk)
                if differences:
                    print(f"Differences found: {differences}")
        
        db_connection.commit()

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        db_connection.rollback()

    finally:
        cursor.close()

In [34]:
import psycopg2

try:
    # Connect to PostgreSQL
    con = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='postgres',
        port=5432,
        dbname='db1'
    )
    cur = con.cursor()

    # Execute queries inside a try block
    query = "SELECT * FROM person"
    cur.execute(query)
    data = cur.fetchall()

    # Process the data here...
    print(data)

    # Commit the transaction if everything is successful
    con.commit()

except psycopg2.Error as e:
    print(f"Error occurred: {e}")
    # Rollback the transaction to clear the error state
    con.rollback()

finally:
    # Always close the connection
    if con:
        cur.close()
        con.close()


[(2, 'jay', 24, 234567), (3, 'Stephanie Wheeler', 41, 1706306), (4, 'Daniel Lamb', 31, 2254623), (5, 'Richard Mills', 27, 1703846), (6, 'Brad Frey', 57, 2025484), (7, 'James Gomez', 27, 2910573), (8, 'Ashley Oconnor', 35, 2420283), (9, 'Jenna Yates', 54, 2720804), (10, 'Brandon Bennett', 33, 1601266), (11, 'Kayla Montgomery', 52, 1237990), (12, 'Jessica Rodriguez', 60, 1060625), (13, 'Nicole Jackson', 31, 2668004), (14, 'Emily Norton', 37, 1460707), (15, 'Paul Taylor', 58, 1566098), (16, 'Charles Zimmerman', 59, 2480175), (17, 'Matthew Navarro', 24, 1000049), (18, 'Spencer Strickland', 41, 1066824), (19, 'Jesse Hendricks', 51, 2147903), (20, 'Alicia Campos', 27, 1261282), (21, 'David Browning', 48, 2201196), (22, 'Danielle Patterson', 25, 1300350), (23, 'Matthew Jackson', 51, 1458019), (24, 'Jason Dunn', 52, 1393517), (25, 'Kimberly Dunlap', 26, 2354808), (26, 'Mikayla Mann', 23, 1063753), (27, 'Johnathan Davis', 34, 1348327), (28, 'Bryan Crawford', 54, 1098997), (29, 'Casey Rice', 33,

In [47]:
import psycopg2

# 1. Reading file in chunks (Adjust if file is CSV, here it's assumed as plain text)
def read_file_in_chunks(file_path, chunk_size=100000):
    with open(file_path, 'r') as file:
        chunk = []
        for i, line in enumerate(file):
            chunk.append(line.strip())  # Strip spaces
            if (i + 1) % chunk_size == 0:
                yield chunk
                chunk = []
        if chunk:
            yield chunk

# 2. Fetching DB data in chunks (PostgreSQL using psycopg2)
def fetch_db_data_in_chunks(cursor, query, chunk_size=100000):
    offset = 0
    while True:
        cursor.execute(query + f" LIMIT {chunk_size} OFFSET {offset}")
        data = cursor.fetchall()
        if not data:
            break
        yield data
        offset += chunk_size

# 3. Comparing file and DB chunks
def compare_file_and_db(file_chunk, db_chunk):
    # Normalize data (strip spaces, convert to lowercase for comparison)
    file_data = set([line.strip().lower() for line in file_chunk])
    db_data = set([str(row[0]).strip().lower() for row in db_chunk])
    
    print(f"File data set: {file_data}")  # Debugging: Check file data
    print(f"DB data set: {db_data}")      # Debugging: Check DB data
    
    return file_data.symmetric_difference(db_data)  # Differences between file and DB

# 4. Combining everything and running the comparison
def compare_large_file_with_db(file_path, db_connection, query):
    cursor = db_connection.cursor()

    try:
        # Iterating over file chunks
        for file_chunk in read_file_in_chunks(file_path):
            print(f"Processing file chunk: {file_chunk}")  # Debugging file chunk
            # Iterating over DB chunks
            for db_chunk in fetch_db_data_in_chunks(cursor, query):
                print(f"Processing DB chunk: {db_chunk}")  # Debugging DB chunk
                
                # Compare the file chunk with the DB chunk
                differences = compare_file_and_db(file_chunk, db_chunk)
                if differences:
                    print(f"Differences found: {differences}")
                else:
                    print("No differences found in this chunk")

        db_connection.commit()  # Commit if everything is successful

    except psycopg2.Error as e:
        print(f"Database error: {e}")
        db_connection.rollback()  # Rollback if there's a failure

    finally:
        cursor.close()

# 5. Running the function (replace with actual file and query)
file_path = 'BIGFILES.txt'
query = "SELECT * FROM person"  # Adjust to your actual query
db_connection = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='postgres',
    port=5432,
    dbname='db1'
)

compare_large_file_with_db(file_path, db_connection, query)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

